In [13]:
#import pandas as pd
#file_path = 'data.txt'

# Read the text file
#data = pd.read_csv(file_path, delimiter="\t", header=None)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
#from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.metrics import confusion_matrix
#import random


In [2]:
# Step 1: Data Preparation
text_data = [
    "This is the first sentence.",
    "Here's the second one.",
    "And the third one follows.",
    # Add more text data as needed
]

text = " ".join(text_data)

In [3]:
# Step 2: Data Preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [4]:
# Step 3: Build the LSTM Model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_length - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

In [5]:
# Step 4: Compile the Model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])


In [7]:
# Step 5: Training the Model
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 0s 18ms/step - loss: 9.0821e-05 - accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 18ms/step - loss: 8.9943e-05 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 17ms/step - loss: 8.9076e-05 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 8.8242e-05 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 8.7397e-05 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 8.6617e-05 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 8.5837e-05 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 8.5078e-05 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 13ms/step - loss: 8.4341e-05 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 18ms/step - loss: 8.3604e-0

In [13]:

# Step 6: Text Generation
seed_text = "Here's the"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')

    predicted = model.predict(token_list, verbose=0)
    predicted_word = tokenizer.index_word[np.argmax(predicted)]

    seed_text += " " + predicted_word

# Print the generated text
print(f"Generated Text: {seed_text}")

Generated Text: Here's the second one follows follows follows follows follows follows follows follows


In [12]:
# Step 7: Test the Accuracy (Perplexity)
test_size = 0.2
split_index = int(len(input_sequences) * (1 - test_size))
X_test, y_test = X[split_index:], y[split_index:]

cross_entropy_sum = 0
word_count = 0

for i in range(len(X_test)):
    input_sequence = X_test[i:i+1]  # Provide a batch of size 1
    target_word = y_test[i]
    predicted = model.predict(input_sequence, verbose=0)
    cross_entropy = -np.log(predicted[0][target_word])
    cross_entropy_sum += cross_entropy
    word_count += 1

perplexity = np.exp(cross_entropy_sum / word_count)

print(f"Test Perplexity: {perplexity}")

Test Perplexity: 1.0000410099885186


Let's say you have a language model that's trained on English text data,
but it's a very poor model. When you evaluate this model on a test dataset,
you might get a perplexity of 1,000 or even 10,000 or more. This means that,
on average, the model is 1,000 or 10,000 times less certain about its predictions compared to a perfect model.

In summary, a high perplexity value, such as 1,000 or higher,
 indicates a poor model that struggles to make accurate predictions,
 while a perplexity close to 1 suggests an excellent model with a high level of accuracy and confidence in its predictions.